<a href="https://colab.research.google.com/github/PandaPowell/ECG_DDA/blob/master/neuropathy_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processing ECG data in Python


1.   Setup
  *   Install and import packages
  *   Mount Google Drive and sync to Git repository
  *   Copy the filtered ECG files from previous R workflow to Google Drive


2.   Processing ECG data
  *  Remove year string from ECG header files
  *  Read signal channels and make sure ECG leads are in channels 1 & 2.
  *  Split ECG data into 10 second snippets, and export plots as images

3.   Next up:
   * Script to train neuropathy classifier



## 1. Setup

### Install and import packages

In [8]:
# Installs
!pip install wfdb
!pip install -U matplotlib

# Imports
from IPython.display import display
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil
import posixpath
import wfdb
import pandas as pd
from scipy import signal
import re
from google.colab import drive


### Mount Google Drive and sync to Git repository

In [ ]:
## Mount google drive
drive.mount('/content/ecg_data')

# Clone git repo (uncomment if not already cloned)
%cd '/content/ecg_data/MyDrive/ecg_ai/'
#! git clone https://github.com/PandaPowell/ECG_DDA

Mounted at /content/ecg_data
/content/ecg_data/MyDrive/ecg_ai


In [ ]:
# Pull updates:
%cd '/content/ecg_data/MyDrive/ecg_ai/ECG_DDA'
! git pull

/content/ecg_data/MyDrive/ecg_ai/ECG_DDA
Already up to date.


### Copy the filtered ECG files from previous R workflow to Google Drive

  * If you've previously run or knitted the `readme.Rmd` script from the *R* session, you can copy the contents of your local `/ecg_data/` folder to the corresponding folder in the repo cloned to your mounted google drive (in this case `/content/ecg_data/MyDrive/ecg_ai/ECG_DDA/ecg_data/`).
  * If you haven't run the *R* preprocessing script, you can grab the preprocessed ECG data from the [`/ecg_data/` folder here](https://drive.google.com/drive/folders/17AX33KuzP2nZwCQ9sWErY_0bSGI-ON1R?usp=sharing)

## 2. Processing ECG data

### Remove year string from ECG header files

Can't read data otherwise (feature or bug?)

In [ ]:
# Function (Credit to Anders Askeland)
def remove_year_from_hea(filename: str) -> None:
    '''
    Removes year from header file in wfdb (.hea) on line 1. Run before reading file with wfdb functions. Does not overwrite original date, wherein the function will rename the original file to: '***_original.hea'.

    PS: Very rudimentary function: will always remove 4 digit strings (regex based) if they appear to the left of newline char (\n).

    Args:
        filepath to .hea file (without .hea extension)
    
    Attributes:
        None
    '''
    
    # Rewrite hea file
    with open(filename + ".hea", "r", encoding='utf-8') as f:
        
        # Read lines
        text = f.readlines()

        # Modify line 1 (with regex)
        pattern = '\s\d{4}(?=(\\n))'
        if not re.search(pattern, text[0]):
            print("Year is already removed. Doing nothing.")
            return(1)
        else:
            print("Removing year from line 1")
            modified_line = re.sub(pattern, '', text[0])
            text[0] = modified_line
    
    # Rename original file (don't need)
    # os.rename(filename + '.hea', filename + '_original.hea')
    
    # Create new file
    with open(filename + ".hea", "w", encoding='utf-8') as f:
        f.writelines(text)


In [ ]:
# Apply function to all .hea files:
ecg_root = '/content/ecg_data/MyDrive/ecg_ai/ECG_DDA/ecg_wfdb/'

hea_list = [val for sublist in [[os.path.join(i[0], j) for j in i[2] if j.endswith('.hea')] for i in os.walk(ecg_root)] for val in sublist]


hea_list_2 = list()

# Remove filename endings and feed to function:
for filename in hea_list:
    name = filename.split('.')[0]
    hea_list_2.append(name)

In [ ]:
for files in hea_list_2:
  remove_year_from_hea(files)

Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Year is already removed. Doing nothing.
Removing year from line 1
Year is already removed. Doing nothing.
Year is already removed. Doing nothing.
Removing year from line 1
Year is already removed. Doing nothing.
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Removing year from line 1
Year is already removed. Doing nothing.
Year is already removed. Doing nothing.
Year is already removed. Doing nothing.
Year is already removed. Doing nothing.
Year

### Read signal channels
Make sure ECG leads are in channels 1 & 2.

Everything is in the right place at the start of monitoring. While that's a good start, wired connections will be prone to the patient unplugging and replugging them to wrong channels in a real-world setting. We'll ignore this issue, as we see no feasible way to address it.

In [9]:
ecg_root = '/content/ecg_data/MyDrive/ecg_ai/ECG_DDA/ecg_wfdb/'


lst = [val for sublist in [[os.path.join(i[0], j) for j in i[2] if j.endswith(('.hea','.dat'))] for i in os.walk(ecg_root)] for val in sublist]

lst2 = []

for filename in lst:
    name = filename.split('.')[0]
    lst2.append(name)

# Get unique list, sorted for keep indices reproducible in case of crashes
lst3 = sorted(list(set(lst2))) 

# Remove image files (in case some splitting to images has been already done)
lst3 = [x for x in lst3 if x.endswith('ECG')]

# Manually check quality of ECGS
# Split into chunks of 10 to avoid running out of memory:
# for filename in lst3[0:9]:
#     print(filename)
#     # load a record using the 'rdrecord' function
#     record = wfdb.rdrecord(filename)    
#     sig1 = pd.DataFrame(record.p_signal)
#     sig1.columns = ['ecg_0' , 'ecg_1', 'sensor_0' , 'sensor_1', 'emg_0', 'emg_1', 'accelerometer_0', 'accelerometer_1']
#     figure, axis = plt.subplots(2, 2, figsize=(25, 8))
#     L = record.fs*10
    
#     axis[0, 0].plot(sig1.iloc[1:L,0])
#     axis[0, 0].set_title(record.record_name)
#     axis[0, 1].plot(sig1.iloc[1:L,1])
#     axis[0, 1].set_title(record.record_name)
#     axis[1, 0].plot(sig1.iloc[2:L,2])
#     axis[1, 0].set_title(record.record_name)
#     axis[1, 1].plot(sig1.iloc[3:L,3]) 
#     axis[1, 1].set_title(record.record_name)


### Split ECG data into 10 second snippets, and export plots as images
While we lose some precision in the data with this approach (depending on image resolution) and lose information contained in the header, this loss is beneficial as it prevents against over-fitting, and the aim is to be able to classify raw ECGs without any additional data.

If Colab throws an error, you may need to restart the run-time and re-run the `Install and import packages` & `Read signal channels` chunks.

#### Sample balance:
Sample length varies a lot: 47 of the 60 individuals produce less than 200 10-second snippets, but 7 individuals produce more than 5000.

To balance the dataset, each individual only contributes the first 2000 seconds (33 minutes, 200 snippets). Hopefully the wires remain in their right places for most of this interval.

In [ ]:
# Plot 10 second ECGs for all individuals, to check sample lengths
# Length varies a lot: 47 of the 60 individuals produce less than 200 10-second snippets,
# but 7 individuals produce more than 5000:

for index, filename in enumerate(lst3):
    
    print('filename: ', filename)
    print('index: ', index)
    
    # load a record using the 'rdrecord' function
    record = wfdb.rdrecord(filename)
    
    sig1 = pd.DataFrame(record.p_signal)
    
    # record time
    print('10-sec snippets: ', sig1.shape[0] / record.fs/ 10, '\n')


#### Export signal 1: lead V1/V2

In [ ]:
# Plot first 200 10 second ECGs for all individuals, both ECG signals

### Signal 1 ###
for index, filename in enumerate(lst3):
    
    
    clear_output(wait=True)
    print('index: ', index)
    print(filename)
    
    # load a record using the 'rdrecord' function
    record = wfdb.rdrecord(filename)
    
    sig1 = pd.DataFrame(record.p_signal)
    
    # record time
    print('How many 10 second measurement intervals:', sig1.shape[0] / record.fs/ 10)
    
    # Define 10 sec interval via record freq times 10
    ten = record.fs*10
    # Create sequence of intervals 
    ten_int = np.arange(0, sig1.shape[0], ten)
    
    # Limit to first 200
    if (len(ten_int) > 200):
        len_sig = 200
    else:
        len_sig = len(ten_int)
        
    print(len_sig)
        
    for x in range(1,len_sig):
        print(x)
        plt.figure(figsize=(15, 2.5), dpi=100)
        plt.plot(sig1.iloc[ten_int[x-1]:ten_int[x],0])
        plt.ylim(-350, 750)
        plt.ylabel('V1/V2  ', rotation=0)
        plt.savefig(filename.replace('ecg_wfdb', 'ecg_img') + '_signal1_' + str(x) + '.png', bbox_inches='tight', dpi=100)
        plt.cla()
        plt.close()



#### Export signal 2: lead V5/V6

In [7]:
### Signal 2 ###
for index, filename in enumerate(lst3):
    
    clear_output(wait=True)
    print('index: ', index)
    print(filename)
    
    # load a record using the 'rdrecord' function
    record = wfdb.rdrecord(filename)
    
    sig1 = pd.DataFrame(record.p_signal)
    
    # record time
    print('How many 10 second measurement intervals:', sig1.shape[0] / record.fs/ 10)
    
    # Define 10 sec interval via record freq times 10
    ten = record.fs*10
    # Create sequence of intervals 
    ten_int = np.arange(0, sig1.shape[0], ten)
    
    # Limit to first 200
    if (len(ten_int) > 200):
        len_sig = 200
    else:
        len_sig = len(ten_int)
        
    print(len_sig)
        
    for x in range(1,len_sig):
        print(x)
        plt.figure(figsize=(15, 2.5), dpi=100)
        plt.plot(sig1.iloc[ten_int[x-1]:ten_int[x],1])
        plt.ylim(-350, 750)
        plt.ylabel('V5/V6  ', rotation=0)
        plt.savefig(filename.replace('ecg_wfdb', 'ecg_img') + '_signal2_' + str(x) + '.png', bbox_inches='tight', dpi=100)
        plt.cla()
        plt.close()
        

index:  59
/content/ecg_data/MyDrive/ecg_ai/ECG_DDA/ecg_wfdb/valid/neuropathy/S0610ECG
How many 10 second measurement intervals: 38.9202
39
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


## 3. Next up: Training a model
Model training is done in the [next notebook](https://colab.research.google.com/drive/1_b-j3hDzYTYbbqGdyUnoaGiPZyRMbcJE?usp=sharing)